In [1]:
#import packages
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import numpy as np
import pandas as pd

In [2]:
#import data
def load_eurosat_data():
    data_dir = 'data/'
    x_train = np.load('x_train.npy')
    y_train = np.load('y_train.npy')
    x_test  = np.load('x_test.npy')
    y_test  = np.load('y_test.npy')
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_eurosat_data()
x_train  = x_train / 255.0
x_test = x_test / 255.0

In [3]:
#build the model
def get_new_model(input_shape):
    """
    Builds a Sequential model compile with the Adam optimiser, sparse categorical cross
    entropy loss function, and a single accuracy metric.
    """
    model = Sequential([
        Conv2D(filters = 16, kernel_size = 3, activation = 'relu', padding = 'SAME', name = 'conv_1', input_shape = input_shape),
        Conv2D(filters = 8, kernel_size = 3, activation = 'relu', padding = 'SAME', name = 'conv_2'),
        MaxPooling2D(pool_size = (8, 8), name = 'pool_1'),
        Flatten(name = 'flatten'),
        Dense(32, activation = 'relu', name = 'dense_1'),
        Dense(10, activation = 'softmax', name = 'dense_2')
    ])
    
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

model = get_new_model(x_train[0].shape) #create the model
model.summary() #model summary

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 64, 64, 16)        448       
_________________________________________________________________
conv_2 (Conv2D)              (None, 64, 64, 8)         1160      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 8, 8, 8)           0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 18,354
Trainable params: 18,354
Non-trainable params: 0
____________________________________________________

In [4]:
#evaluate the model's test accuracy
def get_test_accuracy(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 0)
    print('accuracy: {acc:0.3f}'.format(acc = test_acc))
    
get_test_accuracy(model, x_test, y_test) #accuracy of the initial model

accuracy: 0.104


In [5]:
#create checkpoints to save the model during training, with a criterion
def get_checkpoint_every_epoch():
    """
    Returns a ModelCheckpoint object that saves the weights only at the end of every epoch
    """
    checkpoint_path = 'model_EuroSAT_checkpoints/checkpoint_{epoch:02d}'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, save_weights_only = True,
                                 save_freq = 'epoch', verbose = 1)
    return checkpoint

def get_checkpoint_best_only():
    """
    Returns a ModelCheckpoit object that saves only the weights that generate highest validation (testing) accuracy
    """
    checkpoint_path = 'model_EuroSAT_checkpoint_best/checkpoint'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, save_weights_only = True,
                                 save_best_only = True, save_freq = 'epoch', verbose = 1)
    return checkpoint
    
def get_early_stopping():
    """
    Return an EarlyStopping callback that stops training when the validation accuracy has not improved in the last 3 epochs
    """
    earlystopping = EarlyStopping(monitor = 'val_accuracy', patience = 3, verbose = 1)
    return earlystopping

checkpoint_every_epoch = get_checkpoint_every_epoch()
checkpoint_best_only = get_checkpoint_best_only()
early_stopping = get_early_stopping()

In [6]:
#train model using callbacks
callbacks = [checkpoint_every_epoch, checkpoint_best_only, early_stopping]
model.fit(x_train, y_train, epochs = 50, validation_data = (x_test, y_test), callbacks = callbacks)

Epoch 1/50
125/125 [==============================] - 15s 116ms/step - loss: 2.0291 - accuracy: 0.2177 - val_loss: 1.6131 - val_accuracy: 0.3600

Epoch 00001: saving model to model_EuroSAT_checkpoints\checkpoint_01

Epoch 00001: val_loss improved from inf to 1.61308, saving model to model_EuroSAT_checkpoint_best\checkpoint
Epoch 2/50
125/125 [==============================] - 13s 102ms/step - loss: 1.4643 - accuracy: 0.4250 - val_loss: 1.4312 - val_accuracy: 0.3970

Epoch 00002: saving model to model_EuroSAT_checkpoints\checkpoint_02

Epoch 00002: val_loss improved from 1.61308 to 1.43115, saving model to model_EuroSAT_checkpoint_best\checkpoint
Epoch 3/50
125/125 [==============================] - 13s 104ms/step - loss: 1.3335 - accuracy: 0.4715 - val_loss: 1.3398 - val_accuracy: 0.4600

Epoch 00003: saving model to model_EuroSAT_checkpoints\checkpoint_03

Epoch 00003: val_loss improved from 1.43115 to 1.33983, saving model to model_EuroSAT_checkpoint_best\checkpoint
Epoch 4/50
125/12

In [7]:
#create new nstance of model and load both sets of weights
def get_model_last_epoch(model):
    """
    Creates a new instance of the previously created and loads on the weights from te last training epoch, and return the model
    """
    latest = tf.train.latest_checkpoint('model_EuroSAT_checkpoints')
    model.load_weights(latest)
    return model

def get_model_best_epoch(model):
    """
    Creates a new insance of the previously created CNN and loads on the weights leading to the highest validation accuracy, 
    and return the model
    """
    model.load_weights('model_EuroSAT_checkpoint_best/checkpoint')
    return model

In [8]:
model_last_epoch = get_model_last_epoch(get_new_model(x_train[0].shape))
model_best_epoch = get_model_best_epoch(get_new_model(x_train[0].shape))
print('Model with last epoch weights:')
get_test_accuracy(model_last_epoch, x_test, y_test)
print('')
get_test_accuracy(model_best_epoch, x_test, y_test)

Model with last epoch weights:
accuracy: 0.678

accuracy: 0.678
